# Réalisez une analyse de sentiments grâce au Deep Learning  

L'image de marque d'une entreprise est devenue intrinsèquement liée à sa présence et à sa réputation sur les réseaux sociaux. Les plateformes en ligne offrent une tribune sans précédent pour la diffusion d'opinions, qu'elles soient positives ou négatives.  
La nécessité pour les entreprises de surveiller activement leur présence numérique et d'anticiper les bad buzz est devenue impérative.  

![Air Paradis](https://user.oc-static.com/upload/2023/03/26/16798648691893_Ingenieur-IA-Graphics-P7-01-banner.png) 

Notre objectif est de fournir à [Air Paradis](https://google.com) un outil puissant et prédictif, capable d'analyser les tendances, de détecter les signaux faibles et de permettre une intervention proactive pour prévenir tout impact négatif sur sa réputation en ligne.

## Analyse exploratoire du jeu de données sur l'expression de sentiments de tweets.

### Introduction :  


L'objectif principal de notre projet est de développer un système intelligent capable d'analyser et de comprendre le sentiment d'un message posté par un utilisateur sur un réseau social, en l'occurence ici [Twitter](https://twitter.com/home?lang=fr). En utilisant des techniques avancées de traitement du langage naturel (NLP), notre solution cherchera à extraire des informations clés, à établir des liens sémantiques et à créer une représentation du sentiment de l'utilisateur.

## Nettoyage des données

## Import
Nous utiliserons une stack de Data Science habituelle : `numpy`, `pandas`, `sklearn`, `matplotlib`.

In [1]:
# manipulation des données
import numpy as np
import pandas as pd

# matplotlib et seaborn pour les représentations graphiques
import matplotlib.pyplot as plt
import seaborn as sns
#import missingno as msno

# sklearn preprocessing pour le traiter les variables catégorielles
#from sklearn.preprocessing import LabelEncoder

# Gestion du système de fichiers
import os

# Suppression des alertes
import warnings
warnings.filterwarnings('ignore')

Les données que nous utiliserons durant notre étude ont le chemin d'accès relatif suivant ["../Data"](). Nous pouvons retrouver dans le dossier [./Data]() un fichier csv qui comprend une liste de tweet d'utilisateur. C'est dans ce dossier que nous enregistrerons nos dataframes pour pouvoir les utiliser dans d'autres opérations de notre étude.

In [2]:
print(os.listdir("../Data"))

['2.sample_dataset.csv', '2.sample_dataset2.csv', '1.normalized_dataset.csv', '2.test_df.csv', 'training1600000.csv', '2.cleaned_dataset.csv', '2.train_val_df.csv']


Nous importons le fichier qui nous sera utile pour notre étude.

In [3]:
data = pd.read_csv('../Data/training1600000.csv', encoding='latin-1', header=None)
data = data.rename(columns={data.columns[0]: 'target'})
data = data.rename(columns={data.columns[1]: 'id'})
data = data.rename(columns={data.columns[2]: 'date'})
data = data.rename(columns={data.columns[3]: 'flag'})
data = data.rename(columns={data.columns[4]: 'user'})
data = data.rename(columns={data.columns[5]: 'text'})
data

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


Le fichier contient 1.600.000 tweets d'utilisateurs extrait en utilisant l'API de Twitter. Les tweets ont été annoté (0 = negatif, 4 = positif) et peuvent être utilisé pour détecter le sentiment d'un tweet.

Le DataFrame est caractérisé par 6 variables :  

1. target: le sentiment du tweet (0 = negatif, 4 = positif)
2. id: identifiant du tweet ( 2087)
3. date: date du tweet (Sat May 16 23:58:44 UTC 2009)
4. flag: La requête (LyX). Si il n'y a pas de requête alors la valeur sera NO_QUERY
5. user: l'utilisateur qui a tweeté (robotickilldozr)
6. text: le texte du tweet (avecLyX)

In [4]:
with pd.option_context('display.max_colwidth', None):
    print(data.iloc[0, :])

target                                                                                                                      0
id                                                                                                                 1467810369
date                                                                                             Mon Apr 06 22:19:45 PDT 2009
flag                                                                                                                 NO_QUERY
user                                                                                                          _TheSpecialOne_
text      @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Name: 0, dtype: object


In [5]:
import re
data['text'] = data['text'].apply(lambda x: re.sub(r'\S*@\S*\s?', '', x))
data['text'] = data['text'].apply(lambda x: re.sub(r'http\S+', '', x))
data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data['text'] = data['text'].apply(lambda x: re.sub(r'\b\w\b', '', x))
data['text'] = data['text'].apply(lambda x: re.sub(r'\d', '', x))
data['text'] = data['text'].apply(lambda x: re.sub(r'\s+', ' ', x))
data['text'] = data['text'].apply(lambda x: x.lower())


In [6]:
with pd.option_context('display.max_colwidth', None):
    print(data.iloc[0, :])

target                                                                       0
id                                                                  1467810369
date                                              Mon Apr 06 22:19:45 PDT 2009
flag                                                                  NO_QUERY
user                                                           _TheSpecialOne_
text       awww thats bummer you shoulda got david carr of third day to do it 
Name: 0, dtype: object


In [7]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gaeldelescluse/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
data['text_token'] = 0

for i in range(len(data['text'])):
    data['text_token'][i] = nltk.word_tokenize(data['text'][i])

In [9]:
def remove_stopwords(word_list):
    stop_words = set(stopwords.words('english'))
    filtered_words = []
    for word in word_list:
        if word not in stop_words:
            filtered_words.append(word)
    return filtered_words

In [10]:
for i in range(len(data['text_token'])):
    data['text_token'][i] = remove_stopwords(data['text_token'][i])

In [11]:
with pd.option_context('display.max_colwidth', None):
    print(data.iloc[0, :])

target                                                                           0
id                                                                      1467810369
date                                                  Mon Apr 06 22:19:45 PDT 2009
flag                                                                      NO_QUERY
user                                                               _TheSpecialOne_
text           awww thats bummer you shoulda got david carr of third day to do it 
text_token            [awww, thats, bummer, shoulda, got, david, carr, third, day]
Name: 0, dtype: object


In [12]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('punkt')# segmentation phrases
nltk.download('averaged_perceptron_tagger') # étiquettes grammaticales
nltk.download('wordnet')# synonymes

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gaeldelescluse/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gaeldelescluse/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gaeldelescluse/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
def lemmatize_words(word_list):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, get_pos(word)) for word in word_list]
    return lemmatized_words

def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    return tag_dict.get(tag, wordnet.NOUN)

In [14]:
for i in range(len(data['text_token'])):
    data['text_token'][i] = lemmatize_words(data['text_token'][i])

In [15]:
word_dataset = '../Data/1.normalized_dataset.csv'
data.to_csv(word_dataset, index=False)

In [16]:
df = pd.read_csv("../Data/1.normalized_dataset.csv")

In [17]:
import ast
import re

array_list = df['text_token'].values
data_list = []
for item in array_list:
    data_list.append(ast.literal_eval(item))

df_list = pd.DataFrame({'text_token': data_list})
df = df.drop(columns=['text_token'])
df['text_token'] = df_list['text_token']
df['words'] = df['text_token'].apply(lambda x: ' '.join(x))
df = df.drop(columns=['text_token'])

In [18]:
with pd.option_context('display.max_colwidth', None):
    print(df.iloc[0, :])

target                                                                       0
id                                                                  1467810369
date                                              Mon Apr 06 22:19:45 PDT 2009
flag                                                                  NO_QUERY
user                                                           _TheSpecialOne_
text       awww thats bummer you shoulda got david carr of third day to do it 
words                       awww thats bummer shoulda get david carr third day
Name: 0, dtype: object


In [19]:
word_dataset = '../Data/2.cleaned_dataset.csv'
df.to_csv(word_dataset, index=False)

In [20]:
df = pd.read_csv("../Data/2.cleaned_dataset.csv")

In [21]:
df_neg = df[df['target']== 0].sample(500000)
df_pos = df[df['target']== 4].sample(500000)
df_pos['target'] = 1
liste_concat = [df_neg, df_pos]
df_sample = pd.concat([df_neg, df_pos], ignore_index=True)
df_sample = df_sample.sample(frac=1).reset_index(drop=True)
df_sample

,target,id,date,flag,user,text,words
0,0,2054574443,Sat Jun 06 07:38:48 PDT 2009,NO_QUERY,cantyka,saturday night just at home,saturday night home
1,0,2190983178,Tue Jun 16 04:14:16 PDT 2009,NO_QUERY,untitled01,hiccups wont go away squarespace,hiccup wont go away squarespace
2,1,2052335679,Sat Jun 06 00:23:18 PDT 2009,NO_QUERY,brittnicarter,tonite was aight ready to go to bed,tonite aight ready go bed
3,1,2178121498,Mon Jun 15 07:13:33 PDT 2009,NO_QUERY,NoNo182,busy but good day,busy good day
4,1,2047858869,Fri Jun 05 14:17:18 PDT 2009,NO_QUERY,KidLosoLIVE,NaN,NaN
...,...,...,...,...,...,...,...
999995,1,1999040016,Mon Jun 01 19:32:25 PDT 2009,NO_QUERY,vinnyvero,its totally worth going unvegan for,totally worth go unvegan
999996,0,2256725354,Sat Jun 20 13:12:37 PDT 2009,NO_QUERY,nicroz,forecast rain for the next century bahhh when ...,forecast rain next century bahhh ever go end
999997,0,2237539564,Fri Jun 19 05:58:17 PDT 2009,NO_QUERY,Jaden05,ok so like were having some drama with the sho...,ok like drama show guess dont know there gon n...
999998,1,2187571504,Mon Jun 15 20:28:47 PDT 2009,NO_QUERY,MichelleGrimm,just back from mt olympus wi dells had good da...,back mt olympus wi dell good day kid friend bi...


In [22]:
sample_df = '../Data/2.sample_dataset.csv'
df_sample.to_csv(sample_df, index=False)